In [1]:
import tensorflow as tf
import cv2
from PIL import Image
from scipy import ndimage
import random
import pickle
import pandas as pd
from tqdm import tqdm
import numpy as np

with (open('/kaggle/input/output/imgData.pkl', "rb")) as openfile:
     imgData = pickle.load(openfile)


In [2]:
imgdata_pos = imgData[imgData['cancer'] == 1]
imgdata_neg = imgData[imgData['cancer'] == 0]

imgdata_pos = imgdata_pos.sample(frac = 1)
imgdata_neg = imgdata_neg.sample(frac = 1)

imgdata_neg = imgdata_neg.sample(frac= 0.8)

frames = 10*[imgdata_pos]
frames.append(imgdata_neg)
imgdata_shuff = pd.concat(frames)
imgData = imgdata_shuff.sample(frac=1)

In [3]:
from sklearn.decomposition import PCA
import gc

def random_rotate(imgData):
    #thresh = 0.15
    
    imgData = imgData.reshape(128,128)
    
    
    
    clahe = cv2.createCLAHE(clipLimit=2, tileGridSize=(8,8))
    imgData = cv2.equalizeHist(clahe.apply(imgData))
    
    #pca = PCA(25)
    #imgData = pca.fit_transform(imgData)
    #imgData = pca.inverse_transform(imgData)
    
    
    imgData = ndimage.rotate(imgData, random.randint(-30, 30), reshape=False)

    #imgData = np.clip(imgData,thresh,1)
    imgData = imgData[15:110,15:110]
    
    #imgData = cv2.resize(imgData,(528,528))
    
    
    #imgData = im.fromarray(imgData)
    #imgData = np.asarray(imgData.rotate(random.randint(-20, 20)))
    
    return imgData

In [4]:
imgDataList=[]
for j in tqdm(imgData['img_data']):
    imgDataList.append(random_rotate(j))
imgData['img_data'] = imgDataList

100%|██████████| 54418/54418 [03:34<00:00, 253.84it/s]


In [5]:
from sklearn.preprocessing import MinMaxScaler
train_features= np.array(imgData['img_data'])

#scaler = MinMaxScaler()
#featureTransform = scaler.fit_transform(featureTransform)
#norm_features= []
#for i in range(len(train_features)):
#        norm_features.append(scaler.fit_transform(train_features[i]))
#train_features=np.array(norm_features)


norm_features= []
for i in range(len(train_features)):
        data = train_features[i].reshape(95,95,1)
        norm_features.append(tf.image.grayscale_to_rgb(tf.convert_to_tensor(data)))
train_features=norm_features

2023-02-13 23:06:18.863076: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
imgData['img_data_process'] = " "
imgData['img_data_process'] = train_features
imgData.to_pickle('imgDataNew.pkl' )